# Monotonic Stack
---

最适合求当前元素左边或者右边比它大或者小的元素。

Monotonic - keep the element in the stack increasing or decreasing.

从栈口到栈底：
- 递增：求大的元素
- 递减：求小的元素

单调栈的作用是存放遍历过的元素。

---

#### 739. Daily Temperature

stack 储存的是数组里面的下标。这样好储存遍历过的元素。

T[i] 是我们遍历到的元素。

栈顶元素：T[st.top]

还有一个result数组来记录这个结果。

例子里面T[73, 74, 75, 71, 69, 69, 72, 76, 73]:

1. 第一个和第二个元素T[0] = 73, T[1] = 74, T[2] = 75当遍历到i = 1(第二个元素的时候) st.top = 0：

`result[st.top] = result[0] = i - 0 = 1 - 0 = 1`

2. 弹出T[0] = 73,然后把T[1] = 74加入stack. 这时候遍历下一个元素T[2] i = 2, st.top = 1:

`result[st.top] = result[1] = i - 1 = 2 - 1 = 1`

3. 如果后面继续遍历的元素小于/等于栈里面的元素，就直接吧下标加进栈里面,直到i = 6。st.top = 5,这时候栈里面有：

`stack = [5，4，3，2]`

`result[st.top] = result[5] = i - 5 = 6 - 5 = 1`

4. 这时候弹出 T[5], 发现72还是大于T[4] = 69:

`stack = [4，3，2]`

`result[st.top] = result[4] = i - 4 = 6 - 4 = 2`

5. 同理，72大于T[3] = 71:
`stack = [3，2]`

`result[st.top] = result[3] = i - 3 = 6 - 3 = 3`

6. 72 没有 73 大， 于是放进stack里面， 得到：
`stack = [6，2]`

7. 同理，遍历到i = 7, 76 大于 72，此时i = 7:
`stack = [6，2]`

`result[st.top] = result[6] = i - 6 = 7 - 6 = 1`, 弹出6

`result[st.top] = result[2] = i - 2 = 7 - 2 = 5`，弹出2

8. 这时候T[8] = 73 < 76, 直接加入stack， 这时候stack里面有：

`stack = [8，7]`

这时候不会弹出来了，所以在一开始默认为0就可以了。

---
有个python的用法：

```python
for index, value in enumerate(temperatures):
    print(index, value)

temperatures = [73, 74, 75]
```
输出：
```markdown
0 73
1 74
2 75
```

In [1]:
from typing import List

class Solution:
    def dailyTemperatures(self, temperatures: List[int]) -> List[int]:
        n = len(temperatures)
        res = [0] * n
        stack = []  # 单调递减栈，存下标

        for i, t in enumerate(temperatures):
            while stack and t > temperatures[stack[-1]]:
                j = stack.pop()
                res[j] = i - j
            stack.append(i)
        return res

# example usage
sol = Solution()
print(sol.dailyTemperatures([73, 74, 75, 71, 69, 72, 76, 73]))  # [1, 1, 4, 2, 1, 1, 0, 0]

[1, 1, 4, 2, 1, 1, 0, 0]


#### 496. The Next Greater Element I

思路（栈索引法）(solution1, 3)
- 用**单调递减栈**遍历 `nums2` 的下标，保持栈顶到栈底对应的数值递减。
- 当遇到 `nums2[i]` 大于栈顶下标对应的值时，说明 `nums2[i]` 是栈顶元素的“下一个更大值”，不断弹栈并记录结果。
- 最终栈中剩下的下标没有“更大值”，对应结果为 `-1`。
- 为了按 `nums1` 的顺序输出，需把“每个值在 `nums2` 中的 next greater”先存起来，再按 `nums1` 查结果。

关键优化 (solution2)
- 不要在循环里用 `in nums1` 或 `nums1.index(...)`，否则会退化到 O(n*m)。
- 正确做法：先对 `nums2` 构建 `next_greater_map: 值 -> 下一个更大值`（字典），再用列表推导式按 `nums1` 顺序取。

复杂度
- 时间：O(m + n)，m = len(nums2), n = len(nums1)
- 空间：O(m)


In [4]:
class Solution:
    def nextGreaterElement1(self, nums1: List[int], nums2: List[int]) -> List[int]:
        n = len(nums1)
        m = len(nums2)

        res = [-1] * n
        stack = [0]

        for i in range(1, m):
            if nums2[i] <= nums2[stack[-1]]:
                stack.append(i)

            else:
                while stack and nums2[i] > nums2[stack[-1]]:
                    if nums2[stack[-1]] in nums1:
                        index = nums1.index(nums2[stack[-1]])
                        res[index] = nums2[i]
                    stack.pop()
                stack.append(i)
        
        return res
    
    def nextGreaterElement2(self, nums1: List[int], nums2: List[int]) -> List[int]:
        stack = []  # 存放 nums2的值保持递减
        next_greater = {}   # 映射： 值 -> 下一个更大值

        for x in nums2:
            # 当前 x 更大，结算所有比它小的栈顶元素
            while stack and x > stack[-1]:
                prev = stack.pop()
                next_greater[prev] = x
            stack.append(x)

        # 栈中剩余元素没有更大值
        for v in stack:
            next_greater[v] = -1

        # 按nums1顺序取答案
        return [next_greater[v] for v in nums1]

    def nextGreaterElement3(self, nums1: List[int], nums2: List[int]) -> List[int]:
        n2 = len(nums2)
        st = []                    # 存下标，保持 nums2[st] 单调递减
        next_greater = {}          # 值 -> 下一个更大值

        for i in range(n2):
            while st and nums2[i] > nums2[st[-1]]:
                j = st.pop()
                next_greater[nums2[j]] = nums2[i]
            st.append(i)

        while st:
            j = st.pop()
            next_greater[nums2[j]] = -1

        return [next_greater[v] for v in nums1]

# example usage
sol = Solution()
print(sol.nextGreaterElement1([4, 1, 2], [1, 2, 3, 4]))  # [-1, 2, 3]
print(sol.nextGreaterElement2([4, 1, 2], [1, 2, 3, 4]))  # [-1, 2, 3]
print(sol.nextGreaterElement3([4, 1, 2], [1, 2, 3, 4]))  # [-1, 2, 3]

[-1, 2, 3]
[-1, 2, 3]
[-1, 2, 3]
